<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-mensageria-pubsub-streams/blob/master/redis_workshop_mensageria_pubsub_streams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Mensageria com Listas, Pub/Sub e Streams

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos explorar como utilizar o Redis para mensageria e comunicação eficiente entre serviços. Vamos começar com o conceito de filas, avançando para Pub/Sub e Streams.

Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

## Objetivos do Workshop

Este notebook irá fazer uma introdução ao uso de listas no Redis para implementar filas de mensagens. Em seguida, exploraremos o Pub/Sub e Streams para comunicação em tempo real e processamento de dados.



Espero que gostem! 🖖

## Setup Rápido - e testes pra ver se tá tudo redondo antes de iniciar o lab

In [1]:
# Vamos instalar a lib do redis escolhida para o teste
!pip install -q redis

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 4.6 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,523 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,167 kB]
Get:12 http://security.ubuntu.com/ubuntu

#### Configurando e testando a conexão com o seu Redis Cloud

Coloque o endpoint host, port, e as credenciais pertinentes ao seu setup.

Vou deixar o meu DB mesmo aqui, como referência.

In [2]:
# Testando a redis-cli
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="redis-19581.c308.sa-east-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT=19581
REDIS_PASSWORD="nhtuquVSLbh2kUt2I86z5QwGu3KrcaYx"

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test a Redis connection
!redis-cli $REDIS_CONN PING

# DANGER ZONE - CASO QUEIRA DAR UM REFRESH GERAL
# este comando abaixo ficará comentado, pois ele deleta todos os dados da sua base do Redis Cloud
!redis-cli $REDIS_CONN FLUSHDB

PONG
OK


In [3]:
# Testando via Python (redis-py)
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)
r.ping()

True

# Parte Um - Introdução às Filas no Redis

**Filas são uma estrutura de dados fundamental para a comunicação assíncrona entre serviços.**\

Em ciência da computação, uma fila é uma coleção de entidades mantidas em uma sequência, onde elementos são adicionados em uma extremidade da sequência e removidos da outra. Esta característica é conhecida como FIFO (First In, First Out), o que significa que o primeiro elemento adicionado à fila é o primeiro a ser removido.

Filas são amplamente utilizadas na programação para organizar tarefas de forma sequencial. Um exemplo moderno é um sistema de venda de ingressos de futebol, onde os pedidos são enfileirados e processados na ordem em que são recebidos, garantindo que cada cliente receba seu ingresso corretamente e na sequência certa.


## Criando uma Fila no Redis

No Redis, usamos listas para implementar filas. Vamos começar criando uma fila e adicionando algumas mensagens.

In [4]:
# Deletar a fila caso já exista, só pra não confundir a gente
r.delete("minhafila")

# Adicionar elementos à fila (enqueue)
r.lpush("minhafila", "Tarefa 1")
r.lpush("minhafila", "Tarefa 2")
r.lpush("minhafila", "Tarefa 3")
r.lpush("minhafila", "Tarefa 4")
r.lpush("minhafila", "Tarefa 5")

# Verificar os elementos na fila
print(r.lrange("minhafila", 0, -1))

['Tarefa 5', 'Tarefa 4', 'Tarefa 3', 'Tarefa 2', 'Tarefa 1']


Neste exemplo, usamos o comando LPUSH para adicionar elementos à fila chamada "minhafila". Verificamos os elementos na fila usando o comando LRANGE, que retorna todos os elementos da lista.

## Consumo de Mensagens/Elementos da Fila  (Dequeue)

Depois de adicionar mensagens à fila, precisamos consumir essas mensagens para processá-las.

O Redis oferece o comando `RPOP` para remover e retornar o último elemento da lista, seguindo a ordem **FIFO** (first-in-first-out)

In [7]:
# Remover e consumir o próximo elemento da fila (dequeue)
mensagem = r.rpop("minhafila")
print(f"Mensagem consumida: {mensagem}")

# Verificar os elementos restantes na fila
print(r.lrange("minhafila", 0, -1))

# execute algumas vezes, para vc entender como a fila está sendo consumida e exaurida ao mesmo tempo

Mensagem consumida: Tarefa 3
['Tarefa 5', 'Tarefa 4']


### 🤓 Vamos adicionar um exemplo mais prático e interessante!

Imagine que estamos processando pedidos de ingressos para um jogo de futebol. Cada mensagem na fila representa um pedido de ingresso que precisa ser processado.

In [11]:
import time
# Populando nossa lista com alguns pedidos de tickets novos
r.lpush("tickets:futeborrrr:fila", "Pedido 1",  "Pedido 2",  "Pedido 3",  "Pedido 4",  "Pedido 5",  "Pedido 6", )

##### INICIO DO ROBOZINHO SIMPLES QUE PROCESSA OS PEDIDOS

# Processar pedidos de ingressos
def processar_pedido(pedido):
    time.sleep(0.5)
    print(f"Processando pedido: {pedido}")

# Consumir e processar mensagens na fila
while True:
    pedido = r.rpop("tickets:futeborrrr:fila")
    if not pedido:
        print("Nenhum pedido pendente.")
        break
    processar_pedido(pedido)

# Verificar os elementos restantes na fila
print(r.lrange("tickets:futeborrrr:fila", 0, -1))


Processando pedido: Pedido 1
Processando pedido: Pedido 2
Processando pedido: Pedido 3
Processando pedido: Pedido 4
Processando pedido: Pedido 5
Processando pedido: Pedido 6
Nenhum pedido pendente.
[]


Neste código, definimos uma função processar_pedido que simula o processamento de um pedido. Em seguida, usamos um loop para consumir e processar todas as mensagens na fila "minhafila" até que ela esteja vazia.

Com isso, você aprendeu a consumir mensagens de uma fila no Redis, removendo e processando cada mensagem na ordem correta. Vamos continuar com os próximos passos, como visualizar (peek) e implementar filas confiáveis, em blocos subsequentes.